In [1]:
books = ['Outlander', 'Dragonfly in Amber', 'Voyager', 'Drums of Autumn', 'The Fiery Cross', 'A Breath of Snow and Ashes',
             'An Echo in the Bone', 'Written in My Own Heart’s Blood']
extras = ['Other Books by this Author', 'About the Author']
bookstarts = [50, 17287, 37378, 61857, 89432, 119494, 152540, 177800, 202059]

In [21]:
f = open("data.txt", "r", encoding="utf8")
booknum = 1


book = ''
i = 0
for line in f:
    if i < bookstarts[booknum] and i > bookstarts[booknum-1]:
        book = book + line
    i = i + 1
print(len(book))

1675799


In [4]:
from flair.data import Sentence, Token
from flair.models import SequenceTagger

# load tagger
tagger_ner = SequenceTagger.load("flair/ner-english-ontonotes-fast")
tagger_pos = SequenceTagger.load("flair/pos-english")

2023-05-14 11:47:50,586 SequenceTagger predicts: Dictionary with 75 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-GPE, B-GPE, E-GPE, I-GPE, S-ORG, B-ORG, E-ORG, I-ORG, S-DATE, B-DATE, E-DATE, I-DATE, S-CARDINAL, B-CARDINAL, E-CARDINAL, I-CARDINAL, S-NORP, B-NORP, E-NORP, I-NORP, S-MONEY, B-MONEY, E-MONEY, I-MONEY, S-PERCENT, B-PERCENT, E-PERCENT, I-PERCENT, S-ORDINAL, B-ORDINAL, E-ORDINAL, I-ORDINAL, S-LOC, B-LOC, E-LOC, I-LOC, S-TIME, B-TIME, E-TIME, I-TIME, S-WORK_OF_ART, B-WORK_OF_ART, E-WORK_OF_ART, I-WORK_OF_ART, S-FAC
2023-05-14 11:47:56,308 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


In [5]:
def ner_tag(text):
    sentence_ner = Sentence(text)

    # predict NER tags
    tagger_ner.predict(sentence_ner)
    return sentence_ner

In [6]:
def pos_tag(text):
   sentence_pos = Sentence(text)

   # predict NER tags
   tagger_pos.predict(sentence_pos)
   return sentence_pos

In [7]:
def get_person_before(sentence_ner, token):
    if token.text=='it':
        return token.text
    if token.text=='I' or token.text=='me':
        return 'Claire'
    prev_person = ''
    for entity in sentence_ner.get_spans('ner'):
        if entity.tag == 'PERSON' and entity.start_position<token.start_position:
                prev_person = entity.text
        elif entity.start_position>token.start_position:
            break
    return prev_person

In [34]:
import re
def split_into_chapters(booknum, text):
    if booknum <7:
        return re.split(r'(\\n[0-9]+\\n\\n)', book)
    else:
        # needs refinement
        return re.split(r'[A-Z ]+\\n\\n', book)

In [17]:
def split_into_paragraphs(text):
    splitted = re.split('(\*\*\*)', text)
    return [x for x in splitted if x != '']

In [ ]:
chapters = re.split(r'\n[0-9]+\n\n', book)
i = 0
for chapter in chapters:
    paragraphs = split_into_paragraphs(chapter)
    for paragraph in paragraphs:
        print('Chapter ', i)
        print('Pos tagging....')
        sentence_pos = pos_tag(paragraph)
        print('Ner tagging....')
        sentence_ner = ner_tag(paragraph)
        no_pronouns_book = Sentence(paragraph.split(' '), use_tokenizer=True)
        print('Replacing pronouns....')
        i=0
        for token in sentence_pos:
            if token.tag == 'PRP':
                name = get_person_before(sentence_ner, token)
                new_token = Token(name)
                no_pronouns_book[token.start_position-1].text = name
                no_pronouns_book.tokens[token.idx] = new_token
        with open('data_no_pronouns.txt', 'w') as f:
            f.write(no_pronouns_book.to_original_text())
    i+=1

Chapter  0
Pos tagging....
Ner tagging....
Replacing pronouns....
Chapter  1
Pos tagging....
